In [1]:
import tensorflow as tf

print("NumPy version: ", tf.__version__)

if tf.test.is_gpu_available():
    print("GPU is available")
else:
    print("GPU is not available")
print(tf.config.list_physical_devices('GPU'))


ModuleNotFoundError: No module named 'tensorflow'

In [3]:
!python3.10 -m pip install --upgrade pip
!python3.10 -m pip install --upgrade tensorflow

/bin/bash: python3.10: command not found
/bin/bash: python3.10: command not found


In [3]:
print(1)

1


In [4]:
import logging
import time

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

# conda install matplotlib
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

# https://github.com/sun1638650145/Libraries-and-Extensions-for-TensorFlow-for-Apple-Silicon/releases
# python -m pip install --ignore-installed ~/Downloads/tensorflow_text-2.12.0-cp38-cp38-macosx_11_0_arm64.whl
# import tensorflow_text

### Tensor Constant

In [5]:
num_samples = 10
feature_dim = 3
random_features = np.random.randn(num_samples, feature_dim)
random_labels = np.random.randint(0, 2, size=(num_samples, 1))

features = tf.constant(random_features, dtype=tf.float32)
labels = tf.constant(random_labels, dtype=tf.float32)
dataset = tf.data.Dataset.from_tensor_slices((features, labels))
dataset


<_TensorSliceDataset element_spec=(TensorSpec(shape=(3,), dtype=tf.float32, name=None), TensorSpec(shape=(1,), dtype=tf.float32, name=None))>

In [6]:
features = tf.constant([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 2],
    [1, 1, 2],
    [1, 1, 2],
    [1, 2, 2],
    [1, 2, 2],
    [1, 2, 2],
    [2, 2, 2],
    [2, 2, 2],
    [2, 2, 2],
], dtype=tf.float32)
# higher value means stronger signal from relu
X = tf.fill(dims=(12,3),value=3.)-features
# price: $0.0, $1.0, $2.0
price_labels = tf.constant([
    [0],
    [1],
    [2],
    [0],
    [1],
    [2],
    [0],
    [1],
    [2],
    [0],
    [1],
    [2],
], dtype=tf.int32)
labels = tf.constant([
    [1],
    [1],
    [1],
    [1],
    [1],
    [0],
    [1],
    [0],
    [0],
    [0],
    [0],
    [0],
], dtype=tf.float32)

In [7]:
one_hot_price_labels = tf.one_hot(tf.squeeze(price_labels, axis=-1), depth=3)
print(one_hot_price_labels.shape)
one_hot_result_label = tf.tile(labels, [1,3])
print(one_hot_result_label.shape)

(12, 3)
(12, 3)


In [8]:
# delta prob $0~$1, prob $1~$2, prob $2+
bias_prob = tf.constant([
], dtype=tf.float32)
delta_price_prob = tf.constant([
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
], dtype=tf.float32)
price_prob = tf.math.cumsum(tf.math.softmax(delta_price_prob, axis=-1), axis=-1, reverse=True)
price_prob

<tf.Tensor: shape=(12, 3), dtype=float32, numpy=
array([[1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334]], dtype=float32)>

In [9]:
# delta prob $0~$1, prob $1~$2, prob $2+
bias_prob = tf.constant([
], dtype=tf.float32)
delta_price_prob = tf.constant([
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
    [1.,1.,1.],
], dtype=tf.float32)
price_prob = tf.math.cumsum(tf.math.softmax(delta_price_prob, axis=-1), axis=-1, reverse=True)
price_prob

<tf.Tensor: shape=(12, 3), dtype=float32, numpy=
array([[1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334],
       [1.        , 0.6666667 , 0.33333334]], dtype=float32)>

In [10]:
delta_price_prob

<tf.Tensor: shape=(12, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [11]:
X

<tf.Tensor: shape=(12, 3), dtype=float32, numpy=
array([[2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 2.],
       [2., 2., 1.],
       [2., 2., 1.],
       [2., 2., 1.],
       [2., 1., 1.],
       [2., 1., 1.],
       [2., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [12]:
### CAT_Net

In [19]:
from tensorflow import keras
CLASS_NUM = 3

sample2 = (X, one_hot_result_label, one_hot_price_labels)

class CATCausalModel(tf.keras.Model):
    def __init__(self):
        super(CATCausalModel, self).__init__()
        self.multiply_layer = tf.keras.layers.Multiply()
        self.dense_layer = keras.Sequential([
            keras.layers.Dense(8, activation='relu',),
            keras.layers.Dense(8, activation='relu',),
            # make model compariable, last layer either a sigmoid or a causal activation
            keras.layers.Dense(CLASS_NUM),
        ])
        self.softmax_layer = tf.keras.layers.Softmax(axis=-1)
        self.cumsum_layer = tf.keras.layers.Lambda(
            lambda x: tf.cumsum(x, axis=-1, reverse=True))
        
    def call(self, x, call_mask=None):
        x = self.dense_layer(x)
        x = self.softmax_layer(x)
        x = self.cumsum_layer(x)
#         if call_mask:
#             x = self.multiply_layer(tf.cast(call_mask, tf.float32), x)
        return x

def masked_binary_crossentropy(y_true, y_pred, mask):
    masked_true = tf.boolean_mask(y_true, mask)
    masked_pred = tf.boolean_mask(y_pred, mask)
    loss = tf.keras.losses.binary_crossentropy(masked_true, masked_pred)
    return loss

model = CATCausalModel()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
loss_fn = masked_binary_crossentropy
auc_metrics = tf.keras.metrics.AUC(name='auc')
auprc_metrics = tf.keras.metrics.AUC(name='pr')
num_epochs=10
for epoch in range(num_epochs):
#     print(f"Epoch {epoch+1}/{num_epochs}")
    # Step 3: Perform forward pass
    with tf.GradientTape() as tape:
        # Sample a single training example from the training data
        x_train, y_true, mask = sample2

        # Pass the input features through the model
        y_pred = model(x=x_train)
        # Step 4: Calculate loss
        loss_value = loss_fn(y_true, y_pred, mask)
    
    # Step 5: Backpropagation and optimization
    grads = tape.gradient(loss_value, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    y_true_clip = tf.clip_by_value(y_true, 0, 1)
    y_pred_clip = tf.clip_by_value(y_pred, 0, 1)
    auc_metrics.update_state(y_true_clip, y_pred_clip)
    auc_val = auc_metrics.result().numpy()
    auprc_metrics.update_state(y_true_clip, y_pred_clip)
    auprc_val = auprc_metrics.result().numpy()
    print(f"Loss: {loss_value.numpy()}, AUC: {auc_val}, PR: {auprc_val}")

print(model.predict(X))

Loss: 1.711830496788025, AUC: 0.5555555820465088, PR: 0.5555555820465088
Loss: 1.6799752712249756, AUC: 0.5543981790542603, PR: 0.5543981790542603
Loss: 1.6940438747406006, AUC: 0.5586419701576233, PR: 0.5586419701576233
Loss: 1.6246918439865112, AUC: 0.5584490895271301, PR: 0.5584490895271301
Loss: 1.6533546447753906, AUC: 0.5464814901351929, PR: 0.5464814901351929
Loss: 1.620566725730896, AUC: 0.5482252836227417, PR: 0.5482252836227417
Loss: 1.553983449935913, AUC: 0.5593348145484924, PR: 0.5593348145484924
Loss: 1.5567121505737305, AUC: 0.5709635019302368, PR: 0.5709635019302368
Loss: 1.544515609741211, AUC: 0.5805326700210571, PR: 0.5805326700210571


: 

: 